<a href="https://colab.research.google.com/github/RuslanMavlitov/IDE/blob/master/MATH%26ML_9_Boosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn import tree
from sklearn import model_selection
from sklearn import metrics
from sklearn import ensemble

In [2]:
!pip install xgboost
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.8 MB 1.2 MB/s 


In [3]:
data = pd.read_csv('/content/sample_data/AirPass.zip')

Gender — пол пассажира (женский, мужской);

Customer Type — тип клиента (постоянный/непостоянный клиент);

Age — возраст клиента;

Type of Travel — цель перелета (личная/деловая поездка);

Class — туристический класс пассажира (Business, Eco, Eco Plus);

Flight distance — расстояние полета;

Inflight wifi service — уровень удовлетворённости Wi-Fi (0 — не 
применимо, 1–5);

Departure/Arrival time convenient — уровень удовлетворённости временем отправления и прибытия;

Ease of Online booking — уровень удовлетворённости онлайн-бронированием;

Gate location — уровень удовлетворённости расположением выхода на посадку;

Food and drink — уровень удовлетворённости едой и напитками;

Online boarding — уровень удовлетворённости онлайн-регистрацией;

Seat comfort — уровень удовлетворённости комфортом сидений;

Inflight entertainment — уровень удовлетворённости развлечениями на борту;

On-board service — уровень удовлетворённости сервисом на борту;

Leg room service — уровень удовлетворённости местом для ног;

Baggage handling — уровень удовлетворённости обработкой багажа;

Check-in service — уровень удовлетворённости услугами регистрации;

Inflight service — уровень удовлетворённости обслуживанием во время 
полёта;

Cleanliness — уровень удовлетворённости чистотой;

Departure Delay in Minutes — задержка при отправлении (в минутах);

Arrival Delay in Minutes — задержка при прибытии (в минутах);

Satisfaction — удовлетворённость авиакомпанией — целевая переменная (satisfaction/neutral/dissatisfaction).

In [4]:
data.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


Если при открытии данных первый столбец (порядковый номер строки) считался как отдельный признак, удалите его.

In [7]:
data = data.drop('Unnamed: 0', axis=1)

In [8]:
data.head()

,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


In [10]:
data.describe()

,id,Age,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
count,103904.000000,103904.000000,103904.000000,103904.000000,103904.000000,103904.000000,103904.000000,103904.000000,103904.000000,103904.000000,103904.000000,103904.000000,103904.000000,103904.000000,103904.000000,103904.000000,103904.000000,103904.000000,103594.000000
mean,64924.210502,39.379706,1189.448375,2.729683,3.060296,2.756901,2.976883,3.202129,3.250375,3.439396,3.358158,3.382363,3.351055,3.631833,3.304290,3.640428,3.286351,14.815618,15.178678
std,37463.812252,15.114964,997.147281,1.327829,1.525075,1.398929,1.277621,1.329533,1.349509,1.319088,1.332991,1.288354,1.315605,1.180903,1.265396,1.175663,1.312273,38.230901,38.698682
min,1.000000,7.000000,31.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,32533.750000,27.000000,414.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,3.000000,3.000000,2.000000,0.000000,0.000000
50%,64856.500000,40.000000,843.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000,3.000000,4.000000,3.000000,0.000000,0.000000
75%,97368.250000,51.000000,1743.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,5.000000,4.000000,4.000000,4.000000,5.000000,4.000000,5.000000,4.000000,12.000000,13.000000
max,129880.000000,85.000000,4983.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,1592.000000,1584.000000


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 24 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   id                                 103904 non-null  int64  
 1   Gender                             103904 non-null  object 
 2   Customer Type                      103904 non-null  object 
 3   Age                                103904 non-null  int64  
 4   Type of Travel                     103904 non-null  object 
 5   Class                              103904 non-null  object 
 6   Flight Distance                    103904 non-null  int64  
 7   Inflight wifi service              103904 non-null  int64  
 8   Departure/Arrival time convenient  103904 non-null  int64  
 9   Ease of Online booking             103904 non-null  int64  
 10  Gate location                      103904 non-null  int64  
 11  Food and drink                     1039

##Задание 6.1
Сколько всего в данных пропущенных значений?

In [11]:
data.isnull().sum().sum()

310

##Задание 6.2
давайте избавимся от найденных пропусков. Заполните их все медианными значениями. После этого вычислите среднее арифметическое для признака, отражающего задержку при прибытии в минутах. Ответ округлите до двух знаков после точки-разделителя.

In [17]:
med = data['Arrival Delay in Minutes'].median()
data['Arrival Delay in Minutes'] = data['Arrival Delay in Minutes'].fillna(med)

In [18]:
data['Arrival Delay in Minutes'].mean()

15.133392362180475

##Задание 6.3
Проведём небольшой разведывательный анализ. Посмотрим, в каких категориях пассажиров превалировали удовлетворённые полетом клиенты.

Сравните удовлетворённость полётом мужчин и женщин.

In [27]:
group_1 = data.groupby('Gender')['satisfaction'].value_counts()
group_1

Gender  satisfaction           
Female  neutral or dissatisfied    30193
        satisfied                  22534
Male    neutral or dissatisfied    28686
        satisfied                  22491
Name: satisfaction, dtype: int64

 сравним категории пассажиров в зависимости от их цели поездки.

In [41]:
data.groupby('Type of Travel')['satisfaction'].value_counts()*100/data.shape[0]

Type of Travel   satisfaction           
Business travel  satisfied                  40.177472
                 neutral or dissatisfied    28.785225
Personal Travel  neutral or dissatisfied    27.881506
                 satisfied                   3.155798
Name: satisfaction, dtype: float64

 В каком туристическом классе наибольший процент довольных клиентов

In [40]:
data.groupby('Class')['satisfaction'].value_counts()*100/data.shape[0]

Class     satisfaction           
Business  satisfied                  33.184478
          neutral or dissatisfied    14.614452
Eco       neutral or dissatisfied    36.614567
          satisfied                   8.374076
Eco Plus  neutral or dissatisfied     5.437712
          satisfied                   1.774715
Name: satisfaction, dtype: float64

Перекодируем часть бинарных признаков, чтобы использовать их при обучении:

In [42]:
data['satisfaction'] = data['satisfaction'].map({'neutral or dissatisfied':0 , 'satisfied':1})
data['Customer Type'] = data['Customer Type'].map({'Loyal Customer':1, 'disloyal Customer':0})
data['Type of Travel'] = data['Type of Travel'].map({'Personal Travel':0, 'Business travel':1})

##Задание 6.4
Для остальных категориальных признаков создайте dummy-переменные. Сделайте это с помощью функции get_dummies() из библиотеки Pandas, параметры не меняйте. Сколько теперь признаков в данных (включая целевую переменную)?

In [43]:
data.columns.to_series().groupby(data.dtypes).groups

{int64: ['id', 'Customer Type', 'Age', 'Type of Travel', 'Flight Distance', 'Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking', 'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort', 'Inflight entertainment', 'On-board service', 'Leg room service', 'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness', 'Departure Delay in Minutes', 'satisfaction'], float64: ['Arrival Delay in Minutes'], object: ['Gender', 'Class']}

In [44]:
cat_features = ['Gender', 'Class']

In [45]:
df = pd.get_dummies(data, columns=cat_features)
df.shape[1]

27

##Задание 6.5
Разбейте данные на обучающую и тестовую выборки в соотношении 80/20, параметр random_state = 26. Сколько наблюдений попало в тестовую выборку?

In [48]:
X = df.drop('satisfaction', axis = 1)
Y = df['satisfaction']  
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=0.2, random_state = 26)
Y_test.shape

(20781,)

##Задание 6.6
Теперь нам необходимо реализовать масштабирование данных. Для этого обучите на обучающей выборке метод StandardScaler() и с помощью него преобразуйте и обучающую, и тестовую выборки. Не забудьте, что целевую переменную обрабатывать не нужно.

In [49]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train) 
X_train = scaler.transform(X_train) 
X_test = scaler.transform(X_test) 
X_test[0][0]

0.9408251379303

##Задание 6.7

В качестве первой модели возьмём самую простую — логистическую регрессию. Мы делаем это для того, чтобы потом сравнивать с ней полученные результаты: так вы сможете выяснить, насколько ансамбли смогут улучшить точность прогноза.

Обучите логистическую регрессию с параметрами по умолчанию на наших данных. В качестве ответа введите значение метрики f1_score. Ответ округлите до трёх знаков после точки-разделителя.

0.02879072820657669

##Задание 4.8

Теперь можно перейти к обучению прогностических моделей. Начнём с того, что построим простейшую логистическую регрессию (без настройки гиперпараметров). Это будет та модель, с качеством которой мы будем сравнивать результаты, полученные далее, чтобы оценить превосходство случайного леса над простыми методами.

В качестве ответа введите значение метрики roc_auc на тестовой выборке. Ответ округлите до двух знаков после точки-разделителя.

In [ ]:
from sklearn import linear_model
lr = linear_model.LogisticRegression()
lr.fit(X_train, Y_train)
y_pred_train = lr.predict(X_train)
y_pred_test = lr.predict(X_test)
metrics.roc_auc_score(Y_test, y_pred_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.7261007506246127

##Задание 4.9

Теперь попробуйте обучить на наших данных другой алгоритм — дерево решений. С помощью GridSearchCV сделайте перебор гиперпараметров по следующей сетке:

params = {'max_leaf_nodes': list(range(2, 10)), 'min_samples_split': [2, 3, 4], 'max_depth': [5,7,9,11]}

Для параметра кросс-валидации cv задайте значение 3. Для решающего дерева определите параметр random_state=42. Остальные параметры оставьте по умолчанию.

In [ ]:
params = {'max_leaf_nodes': list(range(5, 15)), 'min_samples_split': [2, 3, 4], 'max_depth': [5,7,9,11]}
grid_search_cv = model_selection.GridSearchCV(tree.DecisionTreeClassifier(random_state=42), params, verbose=3, cv=3)
grid_search_cv.fit(X_train, Y_train)
grid_search_cv.best_params_

Fitting 3 folds for each of 120 candidates, totalling 360 fits
[CV 1/3] END max_depth=5, max_leaf_nodes=5, min_samples_split=2;, score=0.819 total time=   0.2s
[CV 2/3] END max_depth=5, max_leaf_nodes=5, min_samples_split=2;, score=0.820 total time=   0.2s
[CV 3/3] END max_depth=5, max_leaf_nodes=5, min_samples_split=2;, score=0.825 total time=   0.2s
[CV 1/3] END max_depth=5, max_leaf_nodes=5, min_samples_split=3;, score=0.819 total time=   0.2s
[CV 2/3] END max_depth=5, max_leaf_nodes=5, min_samples_split=3;, score=0.820 total time=   0.2s
[CV 3/3] END max_depth=5, max_leaf_nodes=5, min_samples_split=3;, score=0.825 total time=   0.2s
[CV 1/3] END max_depth=5, max_leaf_nodes=5, min_samples_split=4;, score=0.819 total time=   0.2s
[CV 2/3] END max_depth=5, max_leaf_nodes=5, min_samples_split=4;, score=0.820 total time=   0.2s
[CV 3/3] END max_depth=5, max_leaf_nodes=5, min_samples_split=4;, score=0.825 total time=   0.2s
[CV 1/3] END max_depth=5, max_leaf_nodes=6, min_samples_split=2;

{'max_depth': 5, 'max_leaf_nodes': 9, 'min_samples_split': 2}

1. Вычислите значение roc_auc для решающего дерева с гиперпараметрами, определёнными в качестве оптимальных. Ответ округлите до двух знаков после точки-разделителя.

In [ ]:
dt = tree.DecisionTreeClassifier(max_depth = 5, max_leaf_nodes = 9, min_samples_split = 2, random_state=42)
dt.fit(X_train, Y_train)
preds_train = dt.predict(X_train)
preds_test = dt.predict(X_test)
metrics.roc_auc_score(preds_test, Y_test)

0.7915660105378268

2. Какие значения гиперпараметров были выбраны как оптимальные?

In [ ]:
grid_search_cv.best_params_

{'max_depth': 5, 'max_leaf_nodes': 9, 'min_samples_split': 2}

##Задание 4.10

К сожалению, деревья решений не помогли нам в улучшении качества модели, так что попробуем ещё уменьшить ошибку с помощью ансамблей.

Теперь постройте случайный лес, включающий 100 деревьев. Задайте параметр random_state=31. Остальные параметры оставьте по умолчанию.

Какой теперь будет метрика roc_auc на тестовой выборке? Ответ округлите до двух знаков после точки-разделителя.

In [ ]:
rf =  ensemble.RandomForestClassifier(n_estimators = 100, random_state=31)
rf.fit(X_train, Y_train)
preds_train = rf.predict(X_train)
preds_test = rf.predict(X_test)
metrics.roc_auc_score(preds_test, Y_test)

0.8215242568663885

##Задание 4.11

Основные параметры, которые отвечают за качество обучения в случайном лесе, следующие:'max_features', 'min_samples_leaf', 'max_depth'.

Возьмите случайный лес из 100 деревьев и найдите оптимальную комбинацию этих трёх параметров. Сетка для перебора следующая:

{'max_features': [ 4, 5, 6, 7], 'min_samples_leaf': [3, 5, 7, 9, 11], 'max_depth': [5, 10, 15]}

Перебор осуществите с помощью GridSearchCV. Для параметра кросс-валидации cv задайте значение 3. Случайности фиксируйте параметром random_state = 31. Остальные значения оставьте по умолчанию.

Какое значение roc_auc получилось для оптимальных гиперпараметров?

In [ ]:
params = {'max_features': [ 4, 5, 6, 7], 'min_samples_leaf': [3, 5, 7, 9, 11], 'max_depth': [5, 10, 15]}
grid_search_cv = model_selection.GridSearchCV(ensemble.RandomForestClassifier(random_state=31), params, verbose=3, cv=3)
grid_search_cv.fit(X_train, Y_train)
grid_search_cv.best_params_
rf =  ensemble.RandomForestClassifier(n_estimators = 100, max_depth = 15, max_features = 7, min_samples_leaf = 3, random_state=31)
rf.fit(X_train, Y_train)
preds_train = rf.predict(X_train)
preds_test = rf.predict(X_test)
metrics.roc_auc_score(preds_test, Y_test)

Fitting 3 folds for each of 60 candidates, totalling 180 fits
[CV 1/3] END max_depth=5, max_features=4, min_samples_leaf=3;, score=0.776 total time=   1.4s
[CV 2/3] END max_depth=5, max_features=4, min_samples_leaf=3;, score=0.775 total time=   1.7s
[CV 3/3] END max_depth=5, max_features=4, min_samples_leaf=3;, score=0.775 total time=   1.4s
[CV 1/3] END max_depth=5, max_features=4, min_samples_leaf=5;, score=0.776 total time=   1.4s
[CV 2/3] END max_depth=5, max_features=4, min_samples_leaf=5;, score=0.775 total time=   1.4s
[CV 3/3] END max_depth=5, max_features=4, min_samples_leaf=5;, score=0.775 total time=   1.4s
[CV 1/3] END max_depth=5, max_features=4, min_samples_leaf=7;, score=0.776 total time=   1.4s
[CV 2/3] END max_depth=5, max_features=4, min_samples_leaf=7;, score=0.775 total time=   1.4s
[CV 3/3] END max_depth=5, max_features=4, min_samples_leaf=7;, score=0.775 total time=   1.4s
[CV 1/3] END max_depth=5, max_features=4, min_samples_leaf=9;, score=0.776 total time=   1.3

0.8221378962697437

##Задание 4.12

Как мы говорили в предыдущем юните, благодаря случайному лесу можно узнать, какие признаки оказывают большее влияние на целевую переменную по сравнению с другими.

Оцените значимость признаков. Отметьте три признака, которые дают наибольший вклад в целевую переменную:

In [ ]:
features = [x for x in df if x != 'RainTomorrow']
pd.DataFrame({'feat': features,
              'coef': rf.feature_importances_}).sort_values(by='coef', ascending=False)

,feat,coef
7,Humidity3pm,0.250783
2,Rainfall,0.079757
6,Humidity9am,0.070403
10,Cloud9am,0.067092
9,Pressure3pm,0.065272
...,...,...
50,Location_Newcastle,0.000000
62,Location_SalmonGums,0.000000
51,Location_Nhil,0.000000
52,Location_NorahHead,0.000000
